In [4]:
import docx
import pandas as pd
from docx.shared import RGBColor

path = 'C:/Users/jucap/Scripts/Docs/Pediatric Bone Tumors DDx-Video Script.docx' #your docx file path
doc = docx.Document(path)
rosa = RGBColor(255, 0, 255)
naranja = RGBColor(0xff, 0x99, 0x00)
hotspots = []
reviewHotspots = []
locations=[]
allOrange = []
cellIndex = 0


def pinkHotspots(index):
    for text in doc.tables[0]._cells[index].paragraphs:
        if not text.runs:
            pass;
        else:
            if text.runs[0].font.color.rgb == rosa:
                hotspots.append(text.text)
    pass
         
    
    
def orangeHotspots(index):
    for text in doc.tables[0]._cells[index].paragraphs:
        if not text.runs:
            pass;
        else:
            if text.runs[0].font.color.rgb == naranja:
                processRun(text)
            elif len(text.runs) > 1:
                if text.runs[1].font.color.rgb == naranja:
                    processRun(text)
    print(reviewHotspots)
    pass
            
    
    
def processRun(paragraph):
    foundBold = False
    foundItalic = False
    
    reviewHotspots.append("")
    
    for word in paragraph.runs:
        if  (word.bold == True) and (word.text != " "):
            if ( foundBold == False):
                reviewHotspots[-1] = reviewHotspots[-1] + "<b>"
                foundBold = True      
        elif ( foundBold == True ):
            reviewHotspots[-1] = reviewHotspots[-1] + "</b>"
            foundBold = False
            
        if  (word.italic == True) and (word.text != " "):
            if ( foundItalic == False):
                reviewHotspots[-1] = reviewHotspots[-1] + "<i>"
                foundItalic = True          
        elif ( foundItalic == True ):
            reviewHotspots[-1] = reviewHotspots[-1] + "</i>"
            foundItalic = False
              
            
        reviewHotspots[-1] = reviewHotspots[-1] + str(word.text)
    allOrange.append(reviewHotspots[-1])
    if(reviewHotspots[-1][0]!='['):
        reviewHotspots.pop()
        
    return




def organizeHotspots():
    correctedHotspots = []
    correctedHotspots2 = []

    for hp in hotspots:
        correctedHotspots.append(hp[1:])
    for hp in correctedHotspots:
        for index, nhp in reversed(list(enumerate(hp))):
            if nhp == ']':
                correctedHotspots2.append(hp[:index])
                break
    return correctedHotspots2



def organizeHotspots2():
    correctedHotspots = []
    correctedHotspots2 = []
    counter=1
    title=[]
    description=[]
    
    
    for hp in reviewHotspots:
        correctedHotspots.append(hp[1:])
        for index, nhp in list(enumerate(hp)):
            if nhp == ':': 
                title.append(hp[1:index])
    for hp in correctedHotspots:
        for index, nhp in reversed(list(enumerate(hp))):
            if nhp == ']':
                correctedHotspots2.append(hp[:index])
                locations.append(str(counter)+"  "+hp[index:])
                break
        counter= counter+1
    for hp in correctedHotspots2:
        for index, nhp in reversed(list(enumerate(hp))):
            if nhp == ':':
                description.append(hp[index+1:])
                break
    df = pd.DataFrame(list(zip(title, description)),
               columns =['Titles', 'Descriptions'])
    return df



def saveFile():
    fileHotspots = organizeHotspots()
    fileReviewHotspots = organizeHotspots2()
    newLine = ")))"
    newProyect= "Proyecto Nuevo --------"
    with open('C:/Users/jucap/Scripts/Docs/Hotspots.txt', 'a',encoding="utf-8") as f:
        for line in fileHotspots:
            f.write(f"{line}\n")
        f.write(f"{newLine}\n")
    with open('C:/Users/jucap/Scripts/Docs/Positions.txt', 'a',encoding="utf-8") as f:
        f.write(f"{newProyect}\n")
        for line in locations:
            f.write(f"{line}\n")
        f.write(f"\n\n\n\n")
    with open('C:/Users/jucap/Scripts/Docs/ReviewHotspots.txt', 'w',encoding="utf-8") as f:
        for index, line in fileReviewHotspots.iterrows():
            f.write(f"{fileReviewHotspots['Titles'][index]+fileReviewHotspots['Descriptions'][index]}\n")
        f.write(f"\n\n\n\n")
    with open('C:/Users/jucap/Scripts/Docs/allOrange.txt', 'a',encoding="utf-8") as f:
        f.write(f"{newProyect}\n")
        for line in allOrange:
            f.write(f"{line}\n")
        f.write(f"\n\n\n\n")
    
    generateSymbolCard(fileReviewHotspots)
      
    
    
def init():
    for index, cells in list(enumerate(doc.tables[0]._cells)):
        if cells.paragraphs[0].text == "AKA":
            orangeHotspots(index)
            pinkHotspots(index) 
            saveFile()  
            
def generateSymbolCard(dataFrame):
    for index, data in dataFrame.iterrows():
        generateDiv(index,dataFrame['Titles'][index],dataFrame['Descriptions'][index])

def generateDiv(index,tt,dd):
    div1='<div class="t_symbol ui-draggable ui-draggable-handle circle small red"'+\
          'id="t_symbol_'+str(index+1)+'style="left: 495.604px; top: 761.604px" >  '+\
          str(index+1)+ '<div class="t_tooltip_content_wrap left" style="width: auto"><div class="t_tooltip_name">'+\
            tt+'</div><div class="t_tooltip_description">'+dd+'</div></div></div>'
    with open('C:/Users/jucap/Scripts/Docs/Symbols.txt', 'a',encoding="utf-8") as f:
        f.write(f"{div1}\n")
    return div1
init()


['[Asteroid: An <b>osteoid osteoma</b> is a benign, bone-forming tumor that usually involves the proximal femur, tibia, and spine.] → asteroid in sky', '[Sleeping teenage boy: Osteoid osteomas usually present with <b>adolescent boys</b> who complain of bony <b>pain that is worse at night</b> and unrelated to physical activity. → Sleeping teen boy', '[NSAID fire extinguisher: Pain due to osteoid osteomas can usually be managed by <b>NSAIDs</b>. Further treatment depends on the location of the tumor and severity of the symptoms.] → NSAID fire extinguisher', '[Crab eating a chicken bone: An <b>osteosarcoma</b> is the most common primary pediatric bone malignancy that arises from osteoblast activity.] → crab eating chicken bone', '[Long chicken bone: Osteosarcomas occur most frequently at the metaphysis of<b> long bones </b>(e.g. distal femur, proximal tibia, proximal humerus).] → long chicken bone', '[Tender meat: On physical exam, osteosarcomas often present with a <b>tender, soft-tissue